# Using the Crystal Structure Representation of Ward et al.
Recreate the Voronoi-tessellation-based machine learning approach of [Ward et al.](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.96.024104). Builds a model to predict the formation enthalpy based on the crystal structure of a material, using the [FLLA dataset](https://onlinelibrary.wiley.com/doi/abs/10.1002/qua.24917). 

Note: Requires approximately 2 CPU-hours to run. 

*Last Tested Version of matminer*: 0.3.1

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from matminer.datasets import dataframe_loader
from matminer.featurizers.base import MultipleFeaturizer
from matminer.featurizers.composition import ElementProperty, Stoichiometry, ValenceOrbital, IonProperty
from matminer.featurizers.structure import SiteStatsFingerprint, StructuralHeterogeneity, ChemicalOrdering, StructureComposition 
from matminer.featurizers.structure import MaximumPackingEfficiency
from matminer.utils.conversions import dict_to_object
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import ShuffleSplit, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from scipy import stats
from tqdm import tqdm_notebook as tqdm
import numpy as np

## Create the featurizer
Ward et al. use a variety of different featurizers

In [2]:
featurizer = MultipleFeaturizer([
    SiteStatsFingerprint.from_preset("CoordinationNumber_ward-prb-2017"),
    StructuralHeterogeneity(),
    ChemicalOrdering(),
    MaximumPackingEfficiency(),
    SiteStatsFingerprint.from_preset("LocalPropertyDifference_ward-prb-2017"),
    StructureComposition(Stoichiometry()),
    StructureComposition(ElementProperty.from_preset("magpie")),
    StructureComposition(ValenceOrbital(props=['frac'])),
    StructureComposition(IonProperty(fast=True))
])

## Load in a test dataset
Get the dataset from Faber 2015

In [3]:
%%time
data = dataframe_loader.load_flla()
print('Loaded {} entries'.format(len(data)))

Loaded 3938 entries
Wall time: 5.87 s


In [4]:
data['structure'] = dict_to_object(data['structure'])

In [5]:
%%time
print('Total number of features:', len(featurizer.featurize(data['structure'][0])))
print('Number of sites in structure:', len(data['structure'][0]))

Total number of features: 273
Number of sites in structure: 2
Wall time: 119 ms


Ward et al. report 100ms for their average featurization time. At least for this structure, we have a similar runtime.

## Featurize the entire test set
Running the calculations in parallel

In [6]:
%%time
X = featurizer.featurize_many(data['structure'], ignore_errors=True)

Wall time: 22min 17s


Convert `X` to a full array

In [7]:
X = np.array(X)
print('Input data shape:', X.shape)

Input data shape: (3938, 273)


Check how many tessellations failed

In [8]:
failed = np.any(np.isnan(X), axis=1)
print('Number failed: {}/{}'.format(np.sum(failed), len(failed)))

Number failed: 6/3938


# Train an ML Model

In [9]:
model = Pipeline([
    ('imputer', Imputer()), # For the failed structures
    ('model', RandomForestRegressor(n_estimators=150, n_jobs=-1))
])

Train model on whole dataset

In [10]:
%%time
model.fit(X, data['formation_energy_per_atom'])

Wall time: 17.6 s


Pipeline(memory=None,
     steps=[('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('model', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))])

Evaluate the MAE

In [11]:
maes = []
for train_ids, test_ids in tqdm(ShuffleSplit(train_size=3000, n_splits=20).split(X)):
    # Split off the datasets
    train_X = X[train_ids, :]
    train_y = data['formation_energy_per_atom'].iloc[train_ids]
    test_X = X[test_ids, :]
    test_y = data['formation_energy_per_atom'].iloc[test_ids]
    
    # Train the model
    model.fit(train_X, train_y)
    
    # Run the model, compute MAE
    predict_y = model.predict(test_X)
    maes.append(np.abs(test_y - predict_y).mean())

c:\users\logan\miniconda3\lib\site-packages\sklearn\model_selection\_split.py:1639: FutureWarning:

From version 0.21, test_size will always complement train_size unless both are specified.



HBox(children=(IntProgress(value=1, bar_style='info', max=1), HTML(value='')))

In [12]:
print('MAE: {:.3f}+/-{:.3f} eV/atom'.format(np.mean(maes), stats.sem(maes)))

MAE: 0.173+/-0.002 eV/atom


*Finding*: 0.17 eV/atom is in close agreement to what Ward et al. report in their reproduction of this test using OQMD data and Magpie to compute features.